In [ ]:
!apt-get install -y tesseract-ocr
!pip install pytesseract pymupdf pdf2image
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,789 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 126210 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
import requests
import fitz  # PyMuPDF
import pytesseract
from pdf2image import convert_from_path
import os

# Converted Google Drive links to Downloadable file IDs in my drive.
file_ids = [
    "156cEB1WauwkgjaPwPR_D0B3qQcXxGCfF",
    "1RwKejaEbKJVAirFfHPjDKg5F6SpDS5Y5",
    "1tZbvaR66aMJtfmv95v37esLpkyfz-ynl",
    "1mwct2zOZYjR1ld_t5qFSKAmeF8FIDR6a",
    "1pzFBTu6qky8wz2GexXa7oBJgoUKIRhj7"
]

# My directory Folder to save PDFs after downloading from link for processing
pdf_folder = "/content/pdfs"
os.makedirs(pdf_folder, exist_ok=True)

# Download PDFs from Google Drive
def download_pdf(file_id, save_path):
    url = f"https://drive.google.com/uc?id={file_id}"
    response = requests.get(url, stream=True)
    with open(save_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded: {save_path}")

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    all_text = ""

    for page_num in range(len(doc)):
        text = doc[page_num].get_text("text")  # Extract normal text

        # Use OCR if no text is found
        if not text.strip():
            images = convert_from_path(pdf_path, first_page=page_num+1, last_page=page_num+1)
            for img in images:
                text += pytesseract.image_to_string(img)

        all_text += f"\n\n--- Page {page_num + 1} ---\n{text}"

    return all_text

# Process all PDFs
pdf_texts = {}
for file_id in file_ids:
    pdf_path = os.path.join(pdf_folder, f"{file_id}.pdf")
    download_pdf(file_id, pdf_path)
    extracted_text = extract_text_from_pdf(pdf_path)
    pdf_texts[pdf_path] = extracted_text

# Print **all** extracted text
for pdf, text in pdf_texts.items():
    print(f"\n🔹 Extracted from {pdf}:\n{text}\n")

for pdf, text in pdf_texts.items():
    output_file = pdf.replace(".pdf", ".txt")
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(text)
    print(f" Saved full text: {output_file}")



Streaming output truncated to the last 5000 lines.
Successful customer 
relationships
Erratic and anonymous  
customer relationships
Digitalisation and 
connectivity
Communicatively 
networked
Defensive with poor 
communication
In addition to these two scenarios, others are possible that result from various combi­
nations of the megatrends with their different characteristics. It becomes clear that 
strategic management of fashion retail has to involve substantial decisions regarding 
space, customer relationships and the digitalisation of the business model. In order to 
be able to make the best possible decisions about these future-oriented issues, a basis 
for planning is needed. Here, effects that can strengthen other areas and competences 
that have already developed in certain fields and can be generalised must be taken into 
account. In addition, decision-makers should include options and measures that serve 
to ensure acceptance and understanding within the organisations with r

In [ ]:
!pip install nltk textblob spacy wordcloud
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 42.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import re
import spacy
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from textblob import TextBlob
from collections import Counter

nltk.download('punkt')

# Load Spacy NLP model
nlp = spacy.load("en_core_web_sm")

# 🔹 Sample extracted text (Replace this with your PDF extracted data)
pdf_texts = {
    "/content/pdfs/156cEB1WauwkgjaPwPR_D0B3qQcXxGCfF.txt",
    "document_2": "The customer satisfaction was rated 4.5 stars. Issues were reported in logistics.",
    "document_3": "The economy is improving, but inflation is still high. Future predictions are uncertain.",
}

# 🎯 Text Cleaning Function
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W+', ' ', text)  # Remove special characters
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.strip()
    return text

# 🎯 NLP Processing Function
def process_text(text):
    doc = nlp(text)

    # Extract nouns and verbs (important words)
    keywords = [token.text for token in doc if token.pos_ in ["NOUN", "VERB"]]

    # Sentiment Analysis
    sentiment = TextBlob(text).sentiment.polarity  # Range: -1 (negative) to 1 (positive)

    return {
        "cleaned_text": text,
        "keywords": keywords,
        "sentiment": sentiment
    }

# 🔄 Process all PDFs
processed_results = {}
for doc, text in pdf_texts.items():
    cleaned_text = clean_text(text)
    analysis = process_text(cleaned_text)
    processed_results[doc] = analysis

# 📌 Convert Results to DataFrame
df = pd.DataFrame.from_dict(processed_results, orient="index")
print(df)

# 🎨 Word Cloud Visualization
all_text = " ".join(df["cleaned_text"])
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(all_text)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df.to_csv("processed_text_results.csv", index=True)
print("✅ Saved results to CSV!")
